In [1]:
from ase.io import read
from chgnet.graph.converter import CrystalGraphConverter

# === Load .extxyz files ===
train_extxyz = "/home/phanim/harshitrawat/summer/final_work/T3_chgnet_labeled.extxyz"
# valid_extxyz = "/home/phanim/harshitrawat/summer/final_work/T2_chgnet_labeled.extxyz"

train_atoms = read(train_extxyz, index=":")
# valid_atoms = read(valid_extxyz, index=":")

print(f"✅ Loaded {len(train_atoms)} training structures")
# print(f"✅ Loaded {len(valid_atoms)} validation structures")

# === Initialize Converter ===
converter = CrystalGraphConverter(
    atom_graph_cutoff=6.0,
    bond_graph_cutoff=3.0,
    algorithm="fast",
    on_isolated_atoms="warn",
    verbose=True
)

✅ Loaded 1612 training structures
CrystalGraphConverter(algorithm='fast', atom_graph_cutoff=6.0, bond_graph_cutoff=3.0)


In [2]:
from joblib import Parallel, delayed
from pymatgen.io.ase import AseAtomsAdaptor
from chgnet.graph.converter import CrystalGraphConverter

# === Setup ===
ase_adaptor = AseAtomsAdaptor()
converter = CrystalGraphConverter(atom_graph_cutoff=6.0, bond_graph_cutoff=3.0,algorithm="fast",
    on_isolated_atoms="warn",
    verbose=True)

# === Function for one graph ===
def convert_atom(atom):
    structure = ase_adaptor.get_structure(atom)
    return converter(structure)


CrystalGraphConverter(algorithm='fast', atom_graph_cutoff=6.0, bond_graph_cutoff=3.0)


In [3]:

# === Parallel Execution ===
train_graphs = Parallel(n_jobs=32)(delayed(convert_atom)(atom) for atom in train_atoms)
# valid_graphs = Parallel(n_jobs=32)(delayed(convert_atom)(atom) for atom in valid_atoms)


In [4]:
import torch

torch.save(train_graphs, "t3_chgnet_graphs.pt")
# torch.save(valid_graphs, "t2_chgnet_graphs.pt")

print("✅ Saved graphs to disk.")


✅ Saved graphs to disk.


In [5]:
import torch

# === Load validation graphs ===
valid_graphs = torch.load("/home/phanim/harshitrawat/summer/t2_chgnet_graphs.pt")

/tmp/ipykernel_3784347/3064465929.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  valid_graphs = torch.load("/home/phanim/harshitrawat/summer/t2_chgnet_graphs.pt")


In [6]:
from torch.utils.data import Dataset, DataLoader

class GraphDataset(Dataset):
    def __init__(self, graphs):
        self.graphs = graphs

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return self.graphs[idx]

# === Wrap datasets ===
train_dataset = GraphDataset(train_graphs)
valid_dataset = GraphDataset(valid_graphs)


In [7]:
# === Create loaders ===
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

In [8]:
import torch
from chgnet.model.model import CHGNet

# === Force load on CPU first ===
device = torch.device("cuda:0")

# Load the model safely on CPU first
model = CHGNet.load(use_device="cpu", verbose=True)

# Then move manually to the MIG GPU
model = model.to(device)

# === Set to train mode ===
model.train()

# === Optional: Check model arch ===
print("✅ CHGNet loaded on", device)


/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/chgnet/model/model.py:673: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu
✅ CHGNet loaded on cuda:0


In [9]:
valid_extxyz = "/home/phanim/harshitrawat/summer/final_work/T2_chgnet_labeled.extxyz"
valid_atoms = read(valid_extxyz, index=":")


In [10]:
from chgnet.data.dataset import StructureData
import numpy as np
def atoms_to_structuredata(atoms_list):
    structures = []
    energies = []
    forces = []

    for atoms in atoms_list:
        structures.append(ase_adaptor.get_structure(atoms))

        energy = atoms.info.get("REF_energy", None)
        force = atoms.arrays.get("REF_forces", None)

        if energy is None or force is None:
            raise ValueError("Missing REF_energy or REF_forces")

        energies.append(energy)
        forces.append(np.array(force, dtype=np.float32))


    return StructureData(structures=structures, energies=energies, forces=forces)

train_data = atoms_to_structuredata(train_atoms)
valid_data = atoms_to_structuredata(valid_atoms)

print("✅ Created StructureData for training and validation")


StructureData imported 1,612 structures
StructureData imported 705 structures
✅ Created StructureData for training and validation


In [11]:
import torch
from ase.io import write
import joblib

# === Atoms ===
write("t3_atoms.extxyz", train_atoms)

# === StructureData ===
joblib.dump(train_data, "t3_structure_data.pkl")

print("✅ Saved graphs, atoms, and structured datasets")


✅ Saved graphs, atoms, and structured datasets


In [12]:
import os
from chgnet.trainer import Trainer
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
trainer = Trainer(
    model=model,
    targets="ef",           # energy + forces
    optimizer="Adam",
    criterion="MSE",
    learning_rate=1e-2,
    epochs=10,              # you can reduce to 10 for testing
    use_device=device
)


In [13]:
from chgnet.data.dataset import StructureData

# Merge two StructureData objects
full_data = StructureData(
    structures=train_data.structures + valid_data.structures,
    energies=train_data.energies + valid_data.energies,
    forces=train_data.forces + valid_data.forces,
)


StructureData imported 2,317 structures


In [14]:
from chgnet.data.dataset import get_train_val_test_loader
train_loader, val_loader, _ = get_train_val_test_loader(
    dataset=full_data,
    batch_size=4,
    train_ratio=len(train_data) / (len(train_data) + len(valid_data)),
    val_ratio=len(valid_data) / (len(train_data) + len(valid_data)),
    num_workers=4,
    pin_memory=True
)
trainer.train(train_loader, val_loader)

Begin Training: using cuda:0 device
training targets: ef
Epoch: [0][1/403] | Time (3.350)(2.652) | Loss 8147576.5000(8147576.5000) | MAE e 2846.950(2846.950)  f 0.000(0.000)  
Epoch: [0][100/403] | Time (0.884)(0.641) | Loss 13913.1172(4405001.0156) | MAE e 96.950(1741.461)  f 32.460(29.916)  
Epoch: [0][200/403] | Time (0.835)(0.596) | Loss 123764.5312(2232521.3921) | MAE e 264.744(967.267)  f 26.809(29.873)  
Epoch: [0][300/403] | Time (0.848)(0.609) | Loss 35156.8242(1502487.5164) | MAE e 108.872(700.156)  f 36.464(30.479)  
Epoch: [0][400/403] | Time (0.855)(0.619) | Loss 10215.9629(1134728.7083) | MAE e 78.252(560.131)  f 28.263(31.006)  
Val: [100/177] | Time (0.795) | Loss 22565.5059(57673.6318) | MAE e 106.556(192.912)  f 33.040(32.718)  
*   e_MAE (189.346) 	f_MAE (32.769) 	
Epoch: [1][1/403] | Time (6.964)(6.669) | Loss 39943.0391(39943.0391) | MAE e 155.578(155.578)  f 30.701(30.701)  
Epoch: [1][100/403] | Time (0.968)(0.726) | Loss 65660.7266(30864.6952) | MAE e 209.546(14

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fe4c4802020>>
Traceback (most recent call last):
  File "/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
Exception in thread Thread-9 (_pin_memory_loop):
Traceback (most recent call last):
  File "/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/

KeyboardInterrupt: 

In [15]:
import os
from chgnet.trainer import Trainer
from chgnet.data.dataset import get_train_val_test_loader

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

trainer = Trainer(
    model=model,
    targets="ef",           # energy + forces
    optimizer="Adam",
    criterion="MSE",
    learning_rate=1e-3,
    epochs=10,              # you can reduce to 10 for testing
    use_device=device
)

train_loader, val_loader, _ = get_train_val_test_loader(
    dataset=full_data,
    batch_size=4,
    train_ratio=len(train_data) / (len(train_data) + len(valid_data)),
    val_ratio=len(valid_data) / (len(train_data) + len(valid_data)),
    num_workers=4,
    pin_memory=True
)
trainer.train(train_loader, val_loader)

Begin Training: using cuda:0 device
training targets: ef
Epoch: [0][1/403] | Time (7.494)(7.239) | Loss 28128.0000(28128.0000) | MAE e 137.827(137.827)  f 14.108(14.108)  
Epoch: [0][100/403] | Time (0.953)(0.717) | Loss 4919.1357(21626.8521) | MAE e 63.225(125.098)  f 13.551(13.773)  
Epoch: [0][200/403] | Time (0.880)(0.647) | Loss 26392.4531(21444.5587) | MAE e 144.800(124.561)  f 11.996(13.469)  
Epoch: [0][300/403] | Time (0.894)(0.660) | Loss 8416.9648(21028.4198) | MAE e 87.650(123.944)  f 12.484(12.979)  
Epoch: [0][400/403] | Time (0.880)(0.648) | Loss 21321.4492(20758.2915) | MAE e 138.106(123.531)  f 12.685(12.788)  
Val: [100/177] | Time (0.926) | Loss 14084.3691(19420.6850) | MAE e 95.519(118.418)  f 11.843(11.950)  
*   e_MAE (118.149) 	f_MAE (11.974) 	
Epoch: [1][1/403] | Time (7.154)(6.851) | Loss 15058.1543(15058.1543) | MAE e 100.539(100.539)  f 12.546(12.546)  
Epoch: [1][100/403] | Time (0.881)(0.644) | Loss 17774.2871(20570.5069) | MAE e 119.146(121.645)  f 11.111(

In [ ]:
import torch
torch.cuda.empty_cache()

In [2]:
import os
import json
from ase.io import read
from chgnet.model.model import CHGNet
from chgnet.model.dynamics import CHGNetCalculator
import torch

# === Setup ===
folders = {
    "/home/phanim/harshitrawat/summer/md/mdcifs": "/home/phanim/harshitrawat/summer/final_work/mdinfo_chgnetT2_predictions_forces.json",
    "/home/phanim/harshitrawat/summer/md/mdcifs_strained_perturbed": "/home/phanim/harshitrawat/summer/final_work/strain_perturb_chgnetT2_predictions_forces.json"
}

device = torch.device("cuda:0")  # Adjust to MIG slice if needed

# === Load CHGNet ===
model = CHGNet.from_file("/home/phanim/harshitrawat/summer/07-29-2025/bestE_epoch8_e115842_f13049_sNA_mNA.pth.tar")
model = model.to(device)
calc = CHGNetCalculator(model=model, use_device=device)
print("✅ CHGNet model loaded")

def extract_info_from_cif(cif_path):
    try:
        atoms = read(cif_path)
        atoms.calc = calc

        energy = atoms.get_potential_energy()
        forces = atoms.get_forces()
        stress = atoms.get_stress(voigt=False).tolist()

        if forces.shape[0] != len(atoms):
            raise ValueError(f"Force shape mismatch: {forces.shape} vs {len(atoms)}")

        return {
            "file": os.path.basename(cif_path),
            "energy_eV": energy,
            "forces_per_atom_eV_per_A": forces.tolist(),
            "stress_tensor": stress,
            "magmom_total": atoms.get_magnetic_moment() if "magmom" in atoms.arrays else None
        }
    except Exception as e:
        return {
            "file": os.path.basename(cif_path),
            "error": str(e)
        }

# === Label and Save ===
for folder, out_json in folders.items():
    print(f"\n📂 Labeling: {folder}")
    results = []
    cif_files = sorted([f for f in os.listdir(folder) if f.endswith(".cif")])

    for i, fname in enumerate(cif_files):
        full_path = os.path.join(folder, fname)
        result = extract_info_from_cif(full_path)
        results.append(result)
        if "error" in result:
            print(f"❌ {i+1}/{len(cif_files)} — {fname} — {result['error']}")
        else:
            print(f"✅ {i+1}/{len(cif_files)} — {fname}")

    os.makedirs(os.path.dirname(out_json), exist_ok=True)
    with open(out_json, "w") as f:
        json.dump(results, f, indent=2)

    print(f"🧾 Saved {len(results)} entries to: {out_json}")


CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda:0
✅ CHGNet model loaded

📂 Labeling: /home/phanim/harshitrawat/summer/md/mdcifs
✅ 1/6030 — cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0000.cif
✅ 2/6030 — cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0001.cif
✅ 3/6030 — cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0002.cif
✅ 4/6030 — cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0003.cif
✅ 5/6030 — cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0004.cif
✅ 6/6030 — cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0005.cif
✅ 7/6030 — cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0006.cif
✅ 8/6030 — cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0007.cif
✅ 9/6030 — cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0008.cif
✅ 10/6030 — cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0009.cif
✅ 11/6030 — cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_

In [4]:
import os
import json
import numpy as np
import pandas as pd

# === Input Paths ===
ref_paths = [
    "/home/phanim/harshitrawat/summer/final_work/mdinfo_chgnet_predictions_forces.json",
    "/home/phanim/harshitrawat/summer/final_work/strain_perturb_chgnet_predictions_forces.json"
]
t2_paths = [
    "/home/phanim/harshitrawat/summer/final_work/mdinfo_chgnetT2_predictions_forces.json",
    "/home/phanim/harshitrawat/summer/final_work/strain_perturb_chgnetT2_predictions_forces.json"
]

# === Load and combine
def load(pathlist):
    data = []
    for path in pathlist:
        with open(path) as f:
            data += json.load(f)
    return {d["file"]: d for d in data if "error" not in d}

ref_data = load(ref_paths)
t2_data = load(t2_paths)

# === Compare logic
results = []
for file, ref in ref_data.items():
    if file not in t2_data:
        print(f"[WARN] Missing in CHGNet_T2: {file}")
        continue
   

    pred = t2_data[file]

    f_ref = np.array(ref["forces_per_atom_eV_per_A"])
    f_pred = np.array(pred["forces_per_atom_eV_per_A"])

    if f_ref.shape != f_pred.shape:
        print(f"[SKIP] Shape mismatch in {file}: {f_ref.shape} vs {f_pred.shape}")
        continue

    num_atoms = f_ref.shape[0]
    delta_F = f_pred - f_ref

    force_mae = np.mean(np.abs(delta_F))
    force_rmse = np.sqrt(np.mean(delta_F**2))
    force_max = np.max(np.abs(delta_F))

    e_ref = ref["energy_eV"] / num_atoms
    e_pred = pred["energy_eV"] / num_atoms
    delta_E = e_pred - e_ref
    print(f"{file} | e_ref = {e_ref:.6f} | e_pred = {e_pred:.6f} | ΔE = {delta_E:.6f}")
    print(f"   f_ref[0] = {f_ref[0]} | f_pred[0] = {f_pred[0]}")
    results.append({
        "file": file,
        "num_atoms": num_atoms,
        "energy_ref_eV_per_atom": e_ref,
        "energy_t2_eV_per_atom": e_pred,
        "delta_E_eV_per_atom": delta_E,
        "force_mae_eV_per_A": force_mae,
        "force_rmse_eV_per_A": force_rmse,
        "force_max_abs_eV_per_A": force_max
    })

# === Save
out_json = "chgnetT2_vs_universal_comparison.json"
out_excel = "chgnetT2_vs_universal_comparison.xlsx"

with open(out_json, "w") as f:
    json.dump(results, f, indent=2)

df = pd.DataFrame(results)
df.to_excel(out_excel, index=False)

print(f"\n✅ Saved JSON: {out_json}")
print(f"📊 Saved Excel: {out_excel}")

# === Stats
print("\n📊 Global Stats (eV, eV/Å):")
print(f"Energy MAE (per atom): {np.mean(np.abs(df['delta_E_eV_per_atom'])):.5f} eV")
print(f"Energy RMSE (per atom): {np.sqrt(np.mean(df['delta_E_eV_per_atom']**2)):.5f} eV")
print(f"Force MAE: {df['force_mae_eV_per_A'].mean():.5f} eV/Å")
print(f"Force RMSE: {df['force_rmse_eV_per_A'].mean():.5f} eV/Å")
print(f"Max Force Abs Error: {df['force_max_abs_eV_per_A'].max():.5f} eV/Å")


cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0000.cif | e_ref = -4.302460 | e_pred = -2584.395996 | ΔE = -2580.093536
   f_ref[0] = [ 0.00165386  0.00659467 -0.00422581] | f_pred[0] = [15.70427322 60.51914597 90.02623749]
cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0001.cif | e_ref = -4.300037 | e_pred = -2584.396484 | ΔE = -2580.096447
   f_ref[0] = [ 0.34420848  0.10195164 -0.08806539] | f_pred[0] = [13.99886322 65.90104675 87.10457611]
cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0002.cif | e_ref = -4.293908 | e_pred = -2584.398193 | ΔE = -2580.104286
   f_ref[0] = [ 0.57426876  0.23519236 -0.12259344] | f_pred[0] = [ 18.17464447  80.14640045 100.38053894]
cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0003.cif | e_ref = -4.286927 | e_pred = -2584.395020 | ΔE = -2580.108093
   f_ref[0] = [ 0.63648605  0.28764218 -0.13436666] | f_pred[0] = [ 28.71493721  86.31601715 113.29589844]
cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_he

In [23]:
from chgnet.model.model import CHGNet

model = CHGNet.from_file("/home/phanim/harshitrawat/summer/07-29-2025/bestE_epoch8_e115842_f13049_sNA_mNA.pth.tar")


CHGNet v0.3.0 initialized with 412,525 parameters
